### Foundations of Machine Learning EE298
### Assignment No. 2 : Backpropagation
#### Gapuz, Jay

In [1]:
# Load the libraries
import numpy as np
import random
import pandas as pd
np.random.seed(0)
import matplotlib.pyplot as plt

### Create the dataset
Supply the mean and standard deviation of a 1D Gaussian  
For the given mean and std, draw 1,000,000 random samples from the 1D Gaussian  
Build a dataset 𝒟 with 1,000 histogram bins 
  

First create a normal Gaussian distribution, then create 1000 bins  
Create a dataframe of the bin ranges and their frequency(P_x)


In [2]:
# Create a dataset
mean_val = 0
std_val = 0.5

dist = pd.DataFrame(
    np.random.normal(mean_val, std_val, 1000000), 
    columns = ["vals"]
)

# Binning
counts, bin_edges = np.histogram(dist, bins=1000)
print("Total bins: ",len(counts))

# Convert the frequency to P(x):
counts = [i/1000000 for i in counts]
print("sum of P(x): ", np.sum(counts))

# Create a dataframe
bins_arr = []
for i in range(len(bin_edges)):
    try:
        bins_arr.append(
            [bin_edges[i]-0.0000000000001,
             bin_edges[i+1],
             counts[i]]
    )
    except:
        pass  
bins_arr = pd.DataFrame(
    bins_arr, 
    columns = ["low_lim", "high_lim", "p_x"]
)

# Check the bins
bins_arr

Total bins:  1000
sum of P(x):  1.0


,low_lim,high_lim,p_x
0,-2.501149,-2.496290,0.000001
1,-2.496290,-2.491430,0.000000
2,-2.491430,-2.486571,0.000000
3,-2.486571,-2.481711,0.000000
4,-2.481711,-2.476852,0.000000
...,...,...,...
995,2.334034,2.338893,0.000000
996,2.338893,2.343753,0.000000
997,2.343753,2.348612,0.000000
998,2.348612,2.353472,0.000001


From the dataframe containing 1 million samples from a 1D Gaussian,get each sample with the corresponding p(x) from the bins dataframe.   
Code was based here: https://stackoverflow.com/questions/46179362/fastest-way-to-merge-pandas-dataframe-on-ranges

In [3]:
intervals = pd.IntervalIndex.from_arrays(bins_arr.low_lim,bins_arr.high_lim, 'both')
dataset = dist.assign(p_x=bins_arr.set_index(intervals).loc[dist.vals].p_x.values)
dataset.columns = ["x","y"]
dataset

,x,y
0,0.882026,0.000799
1,0.200079,0.003498
2,0.489369,0.002452
3,1.120447,0.000294
4,0.933779,0.000682
...,...,...
999995,0.727953,0.001312
999996,0.054512,0.003895
999997,-0.398641,0.002768
999998,0.437378,0.002686


### Split the dataset to train and test
We will now create a dataset for training and testing.  
We'll use 90 percent for training and 10 percent for testing

In [4]:
train, test = np.split(
    dataset.sample(frac=1, random_state=42), # for reproducibility
    [int(.9*len(dataset))] #70 percent train
)

# Training data
x_train = train["x"].values
x_train = x_train.reshape(len(x_train),1)
y_train = train["y"].values
y_train = y_train.reshape(len(y_train),1)

# Testing data
x_test = test["x"].values
x_test = x_test.reshape(len(x_test),1)
y_test = test["y"].values
y_test = y_test.reshape(len(y_test),1)


### Define the functions that will be used in the implementing the model

This will include the activation function, forward propagation and backward propagation functions using the default Python functions and Numpy

In [5]:
def ReLU(x):
    # this will return values after RelU function is applied
    return np.maximum(x,0)

def ReLUDerivative(x):
    # derivative of the ReLU
    x[x <= 0] = 0
    x[x >  0] = 1
    
    return x

def activateLayer(inputs,weights):
    # multiplying the weights and the input values
    return np.dot(inputs,weights)

def compute_loss(y_true,y_hat):
    # Mean Square Error
    return np.square(y_hat - y_true).sum()

# forward propagation
def forward_propagation(w1,w2,w3,w4,x):
    # Input layer
    h1 = activateLayer(x,w1)
    h1_relu = ReLU(h1)
    # Second layer
    h2 = activateLayer(h1_relu,w2)
    h2_relu = ReLU(h2)
    # Third layer
    h3 = activateLayer(h2_relu,w3)
    # Output layer
    h4 = activateLayer(h3,w4)
    
    return {
        'input_layer': h1,
        'ReLU_1': h1_relu,
        'second_layer': h2,
        'ReLU_2': h2_relu,
        'third_layer': h3,
        'output_layer': h4       
    }   

# backward propagation
def backpropagate(X,y,z4,z3,relu_2,z2,relu_1,z1,w1,w2,w3,w4):
    
    # calculate error on the output 
    grad_z4 = 2 * (z4 - y)
    grad_w4 = np.dot(z3.T,grad_z4)

    # error on the third layer
    grad_z3 = np.dot(grad_z4,w4.T)
    grad_w3 = np.dot(relu_2.T,grad_z3)
    
    # error on the second_layer
    grad_relu_2 = np.dot(grad_z3,w3.T)
    grad_in_relu_2 = grad_relu_2.copy()
    grad_in_relu_2 = ReLUDerivative(grad_in_relu_2)
    grad_z2 = grad_in_relu_2
    grad_w2 = np.dot(relu_1.T,grad_z2)
    
    # error on the first layer
    grad_relu_1 = np.dot(grad_z2,w2.T)
    grad_in_relu_1 = grad_relu_1.copy()
    grad_in_relu_1 = ReLUDerivative(grad_in_relu_1)
    grad_z1 = grad_in_relu_1
    grad_w1 = np.dot(X.T,grad_z1)
    
    return {
        'w1': grad_w1,
        'w2': grad_w2,
        'w3': grad_w3, 
        'w4': grad_w4
    }

def updateWeights(orig_weights,grad_weights,learning_rate):
    orig_weights -= (learning_rate * grad_weights)
    return orig_weights
    

###  Create the training model

Wrap all the functions used and then create the final training model

In [16]:
random.seed(0)
x = x_train
y = y_train


def train_customNeuralNetwork(no_of_epoch = 20,lr = 0.1):
    
    # Randomly initialize weights from a normal distribution
    # with mean of 0 and standard deviation of 0.1
    input_weights  = (0.1**2) * np.random.randn(1,64)
    second_weights = (0.1**2) * np.random.randn(64,64)
    third_weights  = (0.1**2) * np.random.randn(64,64)
    output_weights = (0.1**2) * np.random.randn(64,1)
    
    
    loss_arr = []
    # Train the model:
    for epoch in range(no_of_epoch):
        
        # Perform forward propagation
        fwd_dict = forward_propagation(
            input_weights,
            second_weights,
            third_weights,
            output_weights,
            x
        )
        
        # Compute for the loss
        loss_val = compute_loss(y,fwd_dict.get("output_layer"))
        loss_arr.append(loss_val)
        print("epoch:",epoch + 1, "MSE: ", loss_val)
        
        # Perform backward propagation
        bwd_dict = backpropagate(
            x,y,
            fwd_dict.get("output_layer"),
            fwd_dict.get("third_layer"),
            fwd_dict.get("ReLU_2"),
            fwd_dict.get("second_layer"),
            fwd_dict.get("ReLU_1"),
            fwd_dict.get("input_layer"),
            input_weights,
            second_weights,
            third_weights,
            output_weights
        )
        
        # Update the weights
        input_weights = updateWeights(input_weights, bwd_dict.get('w1'), lr)
        second_weights = updateWeights(second_weights, bwd_dict.get('w2'), lr)
        third_weights = updateWeights(third_weights, bwd_dict.get('w3'), lr)
        output_weights = updateWeights(output_weights, bwd_dict.get('w4'), lr)
        
    return(input_weights,second_weights,third_weights,output_weights)

### Train the network 

In [17]:
# Now call the main routine and train the network
model = train_customNeuralNetwork()

epoch: 1 MSE:  7.820041608503616
epoch: 2 MSE:  516.6032952269815
epoch: 3 MSE:  7.818729069117002
epoch: 4 MSE:  7.818729069117002
epoch: 5 MSE:  7.818729069117002
epoch: 6 MSE:  7.818729069117002
epoch: 7 MSE:  7.818729069117002
epoch: 8 MSE:  7.818729069117002
epoch: 9 MSE:  7.818729069117002
epoch: 10 MSE:  7.818729069117002
epoch: 11 MSE:  7.818729069117002
epoch: 12 MSE:  7.818729069117002
epoch: 13 MSE:  7.818729069117002
epoch: 14 MSE:  7.818729069117002
epoch: 15 MSE:  7.818729069117002
epoch: 16 MSE:  7.818729069117002
epoch: 17 MSE:  7.818729069117002
epoch: 18 MSE:  7.818729069117002
epoch: 19 MSE:  7.818729069117002
epoch: 20 MSE:  7.818729069117002


### Recreate and evaluate the model performance

Call all the weights and then recreate the forward propagation function.
Use the test data and evaluate the overall MSE of the model

In [18]:
test_arr = forward_propagation(
    model[0],
    model[1],
    model[2],
    model[3],
    x_test
)
print("MSE on test data:", compute_loss(test_arr.get('output_layer'),y_test))

MSE on test data: 0.8680629860769999
